In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('../data/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Import OHE class for categorical encoding
from sklearn.preprocessing import OneHotEncoder

def encoder(X):
    ohe = OneHotEncoder() # create OHE instance
    ohe.fit(X) # fit categorical data
    X_enc = ohe.transform(X) # transform and assign to a variable
    
    return X_enc

# Import Standard Scaler class to scale data
from sklearn.preprocessing import StandardScaler

def scaler(X_train, X_test):
    ss = StandardScaler(with_mean=False)    # create SS instance
    X_scaler = ss.fit(X_train)          # fit unscaled data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled    # return scaled data

In [8]:
# Create our target
y = df["loan_status"]

# Create our features
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)

# Transfrom categorical input
X_enc = encoder(X)

In [9]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [10]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_enc, y, random_state=42)

In [11]:
# Scale data with pre-defined Standard Scaler class function
X_train_scaled, X_test_scaled = scaler(X_train, X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# Import classifier class
from imblearn.ensemble import BalancedRandomForestClassifier

# Create the decision tree classifier instance
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=42)

brf.fit(X_train_scaled, y_train)          # fit training data set 
y_pred_brf = brf.predict(X_test_scaled)   # to create predictions

In [13]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred_brf)

0.7007338752119758

In [14]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred_brf)
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"]
)

cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,52,52
Actual low_risk,1685,15416


In [15]:
# Print the imbalanced classification report
y_pred_brf = brf.predict(X_test)

print(classification_report_imbalanced(y_test, y_pred_brf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.00      0.00      1.00      0.00      0.00      0.00       104
   low_risk       0.99      1.00      0.00      1.00      0.00      0.00     17101

avg / total       0.99      0.99      0.01      0.99      0.00      0.00     17205



In [16]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.0009855730417312393, 'num_actv_bc_tl'),
 (0.0005739888699654083, 'policy_code'),
 (0.00020290768954275108, 'loan_amnt'),
 (0.00012645421018137387, 'out_prncp_inv'),
 (0.0, 'verification_status'),
 (0.0, 'total_rev_hi_lim'),
 (0.0, 'total_rec_prncp'),
 (0.0, 'total_rec_late_fee'),
 (0.0, 'total_rec_int'),
 (0.0, 'total_pymnt_inv'),
 (0.0, 'total_pymnt'),
 (0.0, 'total_il_high_credit_limit'),
 (0.0, 'total_cu_tl'),
 (0.0, 'total_bc_limit'),
 (0.0, 'total_bal_il'),
 (0.0, 'total_bal_ex_mort'),
 (0.0, 'total_acc'),
 (0.0, 'tot_hi_cred_lim'),
 (0.0, 'tot_cur_bal'),
 (0.0, 'tot_coll_amt'),
 (0.0, 'tax_liens'),
 (0.0, 'revol_bal'),
 (0.0, 'recoveries'),
 (0.0, 'pymnt_plan'),
 (0.0, 'pub_rec_bankruptcies'),
 (0.0, 'pub_rec'),
 (0.0, 'percent_bc_gt_75'),
 (0.0, 'pct_tl_nvr_dlq'),
 (0.0, 'out_prncp'),
 (0.0, 'open_rv_24m'),
 (0.0, 'open_rv_12m'),
 (0.0, 'open_il_24m'),
 (0.0, 'open_il_12m'),
 (0.0, 'open_act_il'),
 (0.0, 'open_acc_6m'),
 (0.0, 'open_acc'),
 (0.0, 'num_tl_op_past_12m'),
 (0.0

### Easy Ensemble AdaBoost Classifier

In [ ]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier # import class

eec = EasyEnsembleClassifier(
    n_estimators=100, 
    random_state=42, 
    n_jobs=1, 
    sampling_strategy='auto', 
    verbose=0,
    warm_start=False
)

eec.fit(X_train_scaled, y_train)      # fit model 
y_pred = eec.predict(X_test_scaled)   # to predict

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred_eec)

In [ ]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred_eec)
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"]
)

cm_df

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_eec))